In [1]:
import json
import string
import re
import nltk
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train = pd.read_json("train-1.json")
test = pd.read_json("test.json")

In [3]:
df_train = pd.DataFrame(train)
df_test = pd.DataFrame(test)

In [4]:
df_train.head()

,doi,title,abstract,authors,venue,year,references,topics,is_open_access,fields_of_study,citations
0,10.3115/v1/P15-1039,Generating High Quality Proposition Banks for ...,Semantic role labeling (SRL) is crucial to nat...,"[A. Akbik, Laura Chiticariu, Marina Danilevsky...",ACL,2015.0,39,[Semantic role labeling],True,[Computer Science],60
1,10.18653/v1/2020.eval4nlp-1.12,One of these words is not like the other: a re...,Word embeddings are an active topic in the NLP...,"[Jesper Brink Andersen, Mikkel Bak Bertelsen, ...",EVAL4NLP,2020.0,44,[],True,[Computer Science],1
2,10.18653/v1/W17-3516,The Code2Text Challenge: Text Generation in So...,We propose a new shared task for tactical data...,"[Kyle Richardson, Sina Zarrieß, Jonas Kuhn]",INLG,2017.0,30,"[Natural language generation, Library (computi...",True,[Computer Science],5
3,10.18653/v1/S17-2160,The Meaning Factory at SemEval-2017 Task 9: Pr...,We evaluate a semantic parser based on a chara...,"[Rik van Noord, Johan Bos]",SemEval@ACL,2017.0,11,"[Parsing, Convolutional neural network, Text-b...",True,[Computer Science],5
4,10.18653/v1/W15-2205,Semantic Parsing for Textual Entailment,In this paper we gauge the utility of general-...,"[Elisabeth Lien, Milen Kouylekov]",IWPT,2015.0,26,"[Textual entailment, Parsing, SemEval, Semanti...",True,[Computer Science],10


In [5]:
df_train.shape

(9658, 11)

In [6]:
df_train.isnull().sum()

doi                  0
title                0
abstract           159
authors              0
venue                0
year                 3
references           0
topics               0
is_open_access       0
fields_of_study    136
citations            0
dtype: int64

In [7]:
df_train = df_train.dropna()

In [8]:
df_train["authors"] = [",".join(i) for i in df_train["authors"]]
df_train["topics"] = [",".join(i) for i in df_train["topics"]]
df_train["fields_of_study"] = [",".join(i) for i in df_train["fields_of_study"]]

In [9]:
df_train = df_train.replace(r'^\s*$', np.nan, regex=True)

In [10]:
df_train.isnull().sum()

doi                   0
title                 0
abstract              0
authors               0
venue               403
year                  0
references            0
topics             1940
is_open_access        0
fields_of_study       0
citations             0
dtype: int64

In [11]:
df_train = df_train.dropna()

In [12]:
#mode_venue = df_train["venue"].mode().values[0]
#mode_venue

In [13]:
#df_train["venue"] = df_train["venue"].fillna(mode_venue)

In [14]:
def clean_venue(text):
    n = re.sub(r'[0-9]', "", text)
    p = re.sub(r'[^\w\s]', "", n.lower().strip())
    return p

In [15]:
df_train["venue"] = df_train["venue"].apply(lambda x: clean_venue(x))

In [16]:
df_train.groupby(["venue"])["citations"].count().sort_values(ascending=False) #generate ranking percentages

venue
acl              1969
emnlp             892
naacl             563
semeval           423
emnlpijcnlp       175
                 ... 
slpatacl            1
semevalcoling       1
nlpmc               1
scil                1
interspeech         1
Name: citations, Length: 214, dtype: int64

In [17]:
df_train[["venue", "citations"]]

,venue,citations
0,acl,60
2,inlg,5
3,semevalacl,5
4,iwpt,10
5,conll shared task,4
...,...,...
9649,acl,72
9650,repevalacl,49
9651,emnlp,34
9653,dmeacl,8


In [18]:
df_train['rank'] = df_train.groupby('venue')['citations'].rank(pct=True, ascending = False)

In [19]:
df_train['rank_fos'] = df_train.groupby('fields_of_study')['citations'].rank(pct=True, ascending = False)

In [20]:
df_train

,doi,title,abstract,authors,venue,year,references,topics,is_open_access,fields_of_study,citations,rank,rank_fos
0,10.3115/v1/P15-1039,Generating High Quality Proposition Banks for ...,Semantic role labeling (SRL) is crucial to nat...,"A. Akbik,Laura Chiticariu,Marina Danilevsky,Yu...",acl,2015.0,39,Semantic role labeling,True,Computer Science,60,0.232098,0.154924
2,10.18653/v1/W17-3516,The Code2Text Challenge: Text Generation in So...,We propose a new shared task for tactical data...,"Kyle Richardson,Sina Zarrieß,Jonas Kuhn",inlg,2017.0,30,"Natural language generation,Library (computing...",True,Computer Science,5,0.556522,0.729235
3,10.18653/v1/S17-2160,The Meaning Factory at SemEval-2017 Task 9: Pr...,We evaluate a semantic parser based on a chara...,"Rik van Noord,Johan Bos",semevalacl,2017.0,11,"Parsing,Convolutional neural network,Text-base...",True,Computer Science,5,0.558824,0.729235
4,10.18653/v1/W15-2205,Semantic Parsing for Textual Entailment,In this paper we gauge the utility of general-...,"Elisabeth Lien,Milen Kouylekov",iwpt,2015.0,26,"Textual entailment,Parsing,SemEval,Semantic We...",True,Computer Science,10,0.187500,0.564521
5,10.18653/v1/K17-3015,A non-DNN Feature Engineering Approach to Depe...,For this year’s multilingual dependency parsin...,"Xian Qian,Yang Liu",conll shared task,2017.0,12,"Feature engineering,Parsing,Linear classifier,...",True,Computer Science,4,0.704545,0.774918
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9649,10.18653/v1/P18-1241,Attacking Visual Language Grounding with Adver...,Visual language grounding is widely studied in...,"Hongge Chen,Huan Zhang,Pin-Yu Chen,Jinfeng Yi,...",acl,2018.0,44,"Adversary (cryptography),Visual language",True,Computer Science,72,0.193245,0.128449
9650,10.18653/v1/W16-2504,Evaluating Word Embeddings Using a Representat...,Word embeddings are increasingly used in natur...,"Neha Nayak Kennard,Gabor Angeli,Christopher D....",repevalacl,2016.0,15,"Word embedding,Microsoft Word for Mac,Natural ...",True,Computer Science,49,0.300000,0.189336
9651,10.18653/v1/D19-1211,UR-FUNNY: A Multimodal Language Dataset for Un...,Humor is a unique and creative communicative b...,"M. Hasan,Wasifur Rahman,Amir Zadeh,Jianyuan Zh...",emnlp,2019.0,52,"Multimodal interaction,Natural language proces...",True,"Computer Science,Mathematics",34,0.344731,0.323718
9653,10.3115/v1/W14-0202,IBM’s Belief Tracker: Results On Dialog State ...,Accurate dialog state tracking is crucial for ...,"Rudolf Kadlec,Jindřich Libovický,Jan Macek,Jan...",dmeacl,2014.0,25,"Dialog system,Discriminative model,Inclusion B...",True,Computer Science,8,0.111111,0.621329


In [21]:
X = df_train[["rank"]]

In [22]:
y = df_train[["citations"]]

In [23]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

In [25]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

lr.fit(X_train,y_train)

y_pred = lr.predict(X_val)

In [26]:
def score(Y_true, Y_pred):
    y_true = np.log1p(np.maximum(0, Y_true))
    y_pred = np.log1p(np.maximum(0, Y_pred))
    return 1 - np.mean((y_true-y_pred)**2) / np.mean((y_true-np.mean(y_true))**2)

def evaluate(gold_path, pred_path):
    gold = { x['doi']: x['citations'] for x in json.load(open(gold_path)) }
    pred = { x['doi']: x['citations'] for x in json.load(open(pred_path)) }
    y_true = np.array([ gold[key] for key in gold ])
    y_pred = np.array([ pred[key] for key in gold ])
    return score(y_true, y_pred)

In [27]:
score(y_val, y_pred)

citations    0.156218
dtype: float64

In [28]:
from sklearn.metrics import r2_score

r2_score(y_val, y_pred)

0.0924977438819824

In [29]:
#authors ranking based on mean

In [30]:
#citations average
mean_citations = df_train["citations"].mean()

In [31]:
#topic popularity/ranking based on average citation count

In [32]:
#age of an article/number of years since the article was published

In [33]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7099 entries, 0 to 9657
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   doi              7099 non-null   object 
 1   title            7099 non-null   object 
 2   abstract         7099 non-null   object 
 3   authors          7099 non-null   object 
 4   venue            7099 non-null   object 
 5   year             7099 non-null   float64
 6   references       7099 non-null   int64  
 7   topics           7099 non-null   object 
 8   is_open_access   7099 non-null   bool   
 9   fields_of_study  7099 non-null   object 
 10  citations        7099 non-null   int64  
 11  rank             7099 non-null   float64
 12  rank_fos         7099 non-null   float64
dtypes: bool(1), float64(3), int64(2), object(7)
memory usage: 727.9+ KB


In [34]:
df_train["year"] = df_train["year"].astype(int)

In [35]:
df_train

,doi,title,abstract,authors,venue,year,references,topics,is_open_access,fields_of_study,citations,rank,rank_fos
0,10.3115/v1/P15-1039,Generating High Quality Proposition Banks for ...,Semantic role labeling (SRL) is crucial to nat...,"A. Akbik,Laura Chiticariu,Marina Danilevsky,Yu...",acl,2015,39,Semantic role labeling,True,Computer Science,60,0.232098,0.154924
2,10.18653/v1/W17-3516,The Code2Text Challenge: Text Generation in So...,We propose a new shared task for tactical data...,"Kyle Richardson,Sina Zarrieß,Jonas Kuhn",inlg,2017,30,"Natural language generation,Library (computing...",True,Computer Science,5,0.556522,0.729235
3,10.18653/v1/S17-2160,The Meaning Factory at SemEval-2017 Task 9: Pr...,We evaluate a semantic parser based on a chara...,"Rik van Noord,Johan Bos",semevalacl,2017,11,"Parsing,Convolutional neural network,Text-base...",True,Computer Science,5,0.558824,0.729235
4,10.18653/v1/W15-2205,Semantic Parsing for Textual Entailment,In this paper we gauge the utility of general-...,"Elisabeth Lien,Milen Kouylekov",iwpt,2015,26,"Textual entailment,Parsing,SemEval,Semantic We...",True,Computer Science,10,0.187500,0.564521
5,10.18653/v1/K17-3015,A non-DNN Feature Engineering Approach to Depe...,For this year’s multilingual dependency parsin...,"Xian Qian,Yang Liu",conll shared task,2017,12,"Feature engineering,Parsing,Linear classifier,...",True,Computer Science,4,0.704545,0.774918
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9649,10.18653/v1/P18-1241,Attacking Visual Language Grounding with Adver...,Visual language grounding is widely studied in...,"Hongge Chen,Huan Zhang,Pin-Yu Chen,Jinfeng Yi,...",acl,2018,44,"Adversary (cryptography),Visual language",True,Computer Science,72,0.193245,0.128449
9650,10.18653/v1/W16-2504,Evaluating Word Embeddings Using a Representat...,Word embeddings are increasingly used in natur...,"Neha Nayak Kennard,Gabor Angeli,Christopher D....",repevalacl,2016,15,"Word embedding,Microsoft Word for Mac,Natural ...",True,Computer Science,49,0.300000,0.189336
9651,10.18653/v1/D19-1211,UR-FUNNY: A Multimodal Language Dataset for Un...,Humor is a unique and creative communicative b...,"M. Hasan,Wasifur Rahman,Amir Zadeh,Jianyuan Zh...",emnlp,2019,52,"Multimodal interaction,Natural language proces...",True,"Computer Science,Mathematics",34,0.344731,0.323718
9653,10.3115/v1/W14-0202,IBM’s Belief Tracker: Results On Dialog State ...,Accurate dialog state tracking is crucial for ...,"Rudolf Kadlec,Jindřich Libovický,Jan Macek,Jan...",dmeacl,2014,25,"Dialog system,Discriminative model,Inclusion B...",True,Computer Science,8,0.111111,0.621329


In [36]:
df_train["age_of_article"] = [2021 - i for i in df_train['year']]

In [37]:
df_train

,doi,title,abstract,authors,venue,year,references,topics,is_open_access,fields_of_study,citations,rank,rank_fos,age_of_article
0,10.3115/v1/P15-1039,Generating High Quality Proposition Banks for ...,Semantic role labeling (SRL) is crucial to nat...,"A. Akbik,Laura Chiticariu,Marina Danilevsky,Yu...",acl,2015,39,Semantic role labeling,True,Computer Science,60,0.232098,0.154924,6
2,10.18653/v1/W17-3516,The Code2Text Challenge: Text Generation in So...,We propose a new shared task for tactical data...,"Kyle Richardson,Sina Zarrieß,Jonas Kuhn",inlg,2017,30,"Natural language generation,Library (computing...",True,Computer Science,5,0.556522,0.729235,4
3,10.18653/v1/S17-2160,The Meaning Factory at SemEval-2017 Task 9: Pr...,We evaluate a semantic parser based on a chara...,"Rik van Noord,Johan Bos",semevalacl,2017,11,"Parsing,Convolutional neural network,Text-base...",True,Computer Science,5,0.558824,0.729235,4
4,10.18653/v1/W15-2205,Semantic Parsing for Textual Entailment,In this paper we gauge the utility of general-...,"Elisabeth Lien,Milen Kouylekov",iwpt,2015,26,"Textual entailment,Parsing,SemEval,Semantic We...",True,Computer Science,10,0.187500,0.564521,6
5,10.18653/v1/K17-3015,A non-DNN Feature Engineering Approach to Depe...,For this year’s multilingual dependency parsin...,"Xian Qian,Yang Liu",conll shared task,2017,12,"Feature engineering,Parsing,Linear classifier,...",True,Computer Science,4,0.704545,0.774918,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9649,10.18653/v1/P18-1241,Attacking Visual Language Grounding with Adver...,Visual language grounding is widely studied in...,"Hongge Chen,Huan Zhang,Pin-Yu Chen,Jinfeng Yi,...",acl,2018,44,"Adversary (cryptography),Visual language",True,Computer Science,72,0.193245,0.128449,3
9650,10.18653/v1/W16-2504,Evaluating Word Embeddings Using a Representat...,Word embeddings are increasingly used in natur...,"Neha Nayak Kennard,Gabor Angeli,Christopher D....",repevalacl,2016,15,"Word embedding,Microsoft Word for Mac,Natural ...",True,Computer Science,49,0.300000,0.189336,5
9651,10.18653/v1/D19-1211,UR-FUNNY: A Multimodal Language Dataset for Un...,Humor is a unique and creative communicative b...,"M. Hasan,Wasifur Rahman,Amir Zadeh,Jianyuan Zh...",emnlp,2019,52,"Multimodal interaction,Natural language proces...",True,"Computer Science,Mathematics",34,0.344731,0.323718,2
9653,10.3115/v1/W14-0202,IBM’s Belief Tracker: Results On Dialog State ...,Accurate dialog state tracking is crucial for ...,"Rudolf Kadlec,Jindřich Libovický,Jan Macek,Jan...",dmeacl,2014,25,"Dialog system,Discriminative model,Inclusion B...",True,Computer Science,8,0.111111,0.621329,7


In [38]:
col_1 = df_train["citations"]
col_2 = df_train["references"]

In [39]:
col_1.corr(col_2)

0.08249448116257223

In [40]:
new_df = df_train[["references", "is_open_access", "rank", "age_of_article", "citations"]]

In [41]:
new_df = pd.get_dummies(new_df, columns = ["is_open_access"])

In [42]:
new_df

,references,rank,age_of_article,citations,is_open_access_False,is_open_access_True
0,39,0.232098,6,60,0,1
2,30,0.556522,4,5,0,1
3,11,0.558824,4,5,0,1
4,26,0.187500,6,10,0,1
5,12,0.704545,4,4,0,1
...,...,...,...,...,...,...
9649,44,0.193245,3,72,0,1
9650,15,0.300000,5,49,0,1
9651,52,0.344731,2,34,0,1
9653,25,0.111111,7,8,0,1


In [43]:
y1 = new_df[["citations"]]
X1 = new_df[["references", "rank", "age_of_article", "is_open_access_False", "is_open_access_True"]]

In [44]:
X_trains, X_vals, y_trains, y_vals = train_test_split(X1, y1, test_size=0.2, random_state=42)

In [45]:
s = StandardScaler()

X_trains = s.fit_transform(X_trains)
X_vals = s.transform(X_vals)

In [46]:
lreg = LinearRegression()

lreg.fit(X_trains,y_trains)

y_pred_ = lreg.predict(X_vals)

In [47]:
score(y_vals,y_pred_)

citations    0.161786
dtype: float64

In [48]:
authors = df_train[["authors"]]
authors['authors_k'] = authors['authors'].apply(lambda x: " ".join(x.lower() for x in x.split()))
authors['authors_p'] = [x.replace("."," ").split(',') for x in authors['authors_k']]
df_train["authors"] = authors['authors_p']

In [49]:
df_train["authors"] = [",".join(i) for i in df_train["authors"]]

In [50]:
from collections import Counter
def split_and_count(column):
    lis = []
    for i in column:
        for j in i.split(","):
            lis.append(j)
            counts = Counter(lis)
    return counts

In [51]:
df_train["author_counts"] = df_train['authors'].apply(lambda x: len(x.split(',')))

In [52]:
df = pd.DataFrame(list(split_and_count(df_train["authors"]).items()),columns = ['authors','counts']) 

In [53]:
df

,authors,counts
0,a akbik,5
1,laura chiticariu,2
2,marina danilevsky,1
3,yunyao li,3
4,shivakumar vaithyanathan,2
...,...,...
12020,jianyuan zhong,1
12021,md iftekhar tanveer,1
12022,ehsan hoque,1
12023,rudolf kadlec,1


In [54]:
for i in df["authors"]:
    if i in df_train["authors"][0]:
        print(i)

a  akbik
laura chiticariu
marina danilevsky
yunyao li
shivakumar vaithyanathan
huaiyu zhu


In [55]:
for i, j in zip(df["authors"], df["counts"]):
    if i in df_train["authors"][0]:
        print(i,j)

a  akbik 5
laura chiticariu 2
marina danilevsky 1
yunyao li 3
shivakumar vaithyanathan 2
huaiyu zhu 1


In [56]:
def score_2(var1,var2,main_df, index): 
    k = 0
    for i,j in zip(var1,var2):
        if i in main_df[index]:
            #print(j)
            k += j
    return k

score_2(df["authors"], df["counts"],df_train["authors"], 5)

27

In [57]:
df_train.shape

(7099, 15)

In [58]:
len(df_train)

7099

In [59]:
df_train["author_score"] = [score_2(df["authors"], df["counts"],df_train["authors"], each) for each in df_train.index]

In [60]:
df_train

,doi,title,abstract,authors,venue,year,references,topics,is_open_access,fields_of_study,citations,rank,rank_fos,age_of_article,author_counts,author_score
0,10.3115/v1/P15-1039,Generating High Quality Proposition Banks for ...,Semantic role labeling (SRL) is crucial to nat...,"a akbik,laura chiticariu,marina danilevsky,yu...",acl,2015,39,Semantic role labeling,True,Computer Science,60,0.232098,0.154924,6,6,14
2,10.18653/v1/W17-3516,The Code2Text Challenge: Text Generation in So...,We propose a new shared task for tactical data...,"kyle richardson,sina zarrieß,jonas kuhn",inlg,2017,30,"Natural language generation,Library (computing...",True,Computer Science,5,0.556522,0.729235,4,3,21
3,10.18653/v1/S17-2160,The Meaning Factory at SemEval-2017 Task 9: Pr...,We evaluate a semantic parser based on a chara...,"rik van noord,johan bos",semevalacl,2017,11,"Parsing,Convolutional neural network,Text-base...",True,Computer Science,5,0.558824,0.729235,4,2,8
4,10.18653/v1/W15-2205,Semantic Parsing for Textual Entailment,In this paper we gauge the utility of general-...,"elisabeth lien,milen kouylekov",iwpt,2015,26,"Textual entailment,Parsing,SemEval,Semantic We...",True,Computer Science,10,0.187500,0.564521,6,2,5
5,10.18653/v1/K17-3015,A non-DNN Feature Engineering Approach to Depe...,For this year’s multilingual dependency parsin...,"xian qian,yang liu",conll shared task,2017,12,"Feature engineering,Parsing,Linear classifier,...",True,Computer Science,4,0.704545,0.774918,4,2,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9649,10.18653/v1/P18-1241,Attacking Visual Language Grounding with Adver...,Visual language grounding is widely studied in...,"hongge chen,huan zhang,pin-yu chen,jinfeng yi,...",acl,2018,44,"Adversary (cryptography),Visual language",True,Computer Science,72,0.193245,0.128449,3,5,8
9650,10.18653/v1/W16-2504,Evaluating Word Embeddings Using a Representat...,Word embeddings are increasingly used in natur...,"neha nayak kennard,gabor angeli,christopher d ...",repevalacl,2016,15,"Word embedding,Microsoft Word for Mac,Natural ...",True,Computer Science,49,0.300000,0.189336,5,3,37
9651,10.18653/v1/D19-1211,UR-FUNNY: A Multimodal Language Dataset for Un...,Humor is a unique and creative communicative b...,"m hasan,wasifur rahman,amir zadeh,jianyuan zh...",emnlp,2019,52,"Multimodal interaction,Natural language proces...",True,"Computer Science,Mathematics",34,0.344731,0.323718,2,7,18
9653,10.3115/v1/W14-0202,IBM’s Belief Tracker: Results On Dialog State ...,Accurate dialog state tracking is crucial for ...,"rudolf kadlec,jindřich libovický,jan macek,jan...",dmeacl,2014,25,"Dialog system,Discriminative model,Inclusion B...",True,Computer Science,8,0.111111,0.621329,7,4,10


In [61]:
df_train['author_rank'] = df_train.groupby('author_score')['citations'].rank(pct=True, ascending = True)

In [62]:
df_train

,doi,title,abstract,authors,venue,year,references,topics,is_open_access,fields_of_study,citations,rank,rank_fos,age_of_article,author_counts,author_score,author_rank
0,10.3115/v1/P15-1039,Generating High Quality Proposition Banks for ...,Semantic role labeling (SRL) is crucial to nat...,"a akbik,laura chiticariu,marina danilevsky,yu...",acl,2015,39,Semantic role labeling,True,Computer Science,60,0.232098,0.154924,6,6,14,0.879397
2,10.18653/v1/W17-3516,The Code2Text Challenge: Text Generation in So...,We propose a new shared task for tactical data...,"kyle richardson,sina zarrieß,jonas kuhn",inlg,2017,30,"Natural language generation,Library (computing...",True,Computer Science,5,0.556522,0.729235,4,3,21,0.200000
3,10.18653/v1/S17-2160,The Meaning Factory at SemEval-2017 Task 9: Pr...,We evaluate a semantic parser based on a chara...,"rik van noord,johan bos",semevalacl,2017,11,"Parsing,Convolutional neural network,Text-base...",True,Computer Science,5,0.558824,0.729235,4,2,8,0.280000
4,10.18653/v1/W15-2205,Semantic Parsing for Textual Entailment,In this paper we gauge the utility of general-...,"elisabeth lien,milen kouylekov",iwpt,2015,26,"Textual entailment,Parsing,SemEval,Semantic We...",True,Computer Science,10,0.187500,0.564521,6,2,5,0.617560
5,10.18653/v1/K17-3015,A non-DNN Feature Engineering Approach to Depe...,For this year’s multilingual dependency parsin...,"xian qian,yang liu",conll shared task,2017,12,"Feature engineering,Parsing,Linear classifier,...",True,Computer Science,4,0.704545,0.774918,4,2,27,0.159091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9649,10.18653/v1/P18-1241,Attacking Visual Language Grounding with Adver...,Visual language grounding is widely studied in...,"hongge chen,huan zhang,pin-yu chen,jinfeng yi,...",acl,2018,44,"Adversary (cryptography),Visual language",True,Computer Science,72,0.193245,0.128449,3,5,8,0.908000
9650,10.18653/v1/W16-2504,Evaluating Word Embeddings Using a Representat...,Word embeddings are increasingly used in natur...,"neha nayak kennard,gabor angeli,christopher d ...",repevalacl,2016,15,"Word embedding,Microsoft Word for Mac,Natural ...",True,Computer Science,49,0.300000,0.189336,5,3,37,0.580000
9651,10.18653/v1/D19-1211,UR-FUNNY: A Multimodal Language Dataset for Un...,Humor is a unique and creative communicative b...,"m hasan,wasifur rahman,amir zadeh,jianyuan zh...",emnlp,2019,52,"Multimodal interaction,Natural language proces...",True,"Computer Science,Mathematics",34,0.344731,0.323718,2,7,18,0.702454
9653,10.3115/v1/W14-0202,IBM’s Belief Tracker: Results On Dialog State ...,Accurate dialog state tracking is crucial for ...,"rudolf kadlec,jindřich libovický,jan macek,jan...",dmeacl,2014,25,"Dialog system,Discriminative model,Inclusion B...",True,Computer Science,8,0.111111,0.621329,7,4,10,0.451754


In [66]:
df_train = pd.get_dummies(df_train, columns=["is_open_access"])
df_train

,doi,title,abstract,authors,venue,year,references,topics,fields_of_study,citations,rank,rank_fos,age_of_article,author_counts,author_score,author_rank,is_open_access_False,is_open_access_True
0,10.3115/v1/P15-1039,Generating High Quality Proposition Banks for ...,Semantic role labeling (SRL) is crucial to nat...,"a akbik,laura chiticariu,marina danilevsky,yu...",acl,2015,39,Semantic role labeling,Computer Science,60,0.232098,0.154924,6,6,14,0.879397,0,1
2,10.18653/v1/W17-3516,The Code2Text Challenge: Text Generation in So...,We propose a new shared task for tactical data...,"kyle richardson,sina zarrieß,jonas kuhn",inlg,2017,30,"Natural language generation,Library (computing...",Computer Science,5,0.556522,0.729235,4,3,21,0.200000,0,1
3,10.18653/v1/S17-2160,The Meaning Factory at SemEval-2017 Task 9: Pr...,We evaluate a semantic parser based on a chara...,"rik van noord,johan bos",semevalacl,2017,11,"Parsing,Convolutional neural network,Text-base...",Computer Science,5,0.558824,0.729235,4,2,8,0.280000,0,1
4,10.18653/v1/W15-2205,Semantic Parsing for Textual Entailment,In this paper we gauge the utility of general-...,"elisabeth lien,milen kouylekov",iwpt,2015,26,"Textual entailment,Parsing,SemEval,Semantic We...",Computer Science,10,0.187500,0.564521,6,2,5,0.617560,0,1
5,10.18653/v1/K17-3015,A non-DNN Feature Engineering Approach to Depe...,For this year’s multilingual dependency parsin...,"xian qian,yang liu",conll shared task,2017,12,"Feature engineering,Parsing,Linear classifier,...",Computer Science,4,0.704545,0.774918,4,2,27,0.159091,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9649,10.18653/v1/P18-1241,Attacking Visual Language Grounding with Adver...,Visual language grounding is widely studied in...,"hongge chen,huan zhang,pin-yu chen,jinfeng yi,...",acl,2018,44,"Adversary (cryptography),Visual language",Computer Science,72,0.193245,0.128449,3,5,8,0.908000,0,1
9650,10.18653/v1/W16-2504,Evaluating Word Embeddings Using a Representat...,Word embeddings are increasingly used in natur...,"neha nayak kennard,gabor angeli,christopher d ...",repevalacl,2016,15,"Word embedding,Microsoft Word for Mac,Natural ...",Computer Science,49,0.300000,0.189336,5,3,37,0.580000,0,1
9651,10.18653/v1/D19-1211,UR-FUNNY: A Multimodal Language Dataset for Un...,Humor is a unique and creative communicative b...,"m hasan,wasifur rahman,amir zadeh,jianyuan zh...",emnlp,2019,52,"Multimodal interaction,Natural language proces...","Computer Science,Mathematics",34,0.344731,0.323718,2,7,18,0.702454,0,1
9653,10.3115/v1/W14-0202,IBM’s Belief Tracker: Results On Dialog State ...,Accurate dialog state tracking is crucial for ...,"rudolf kadlec,jindřich libovický,jan macek,jan...",dmeacl,2014,25,"Dialog system,Discriminative model,Inclusion B...",Computer Science,8,0.111111,0.621329,7,4,10,0.451754,0,1


In [68]:
X_author = df_train[["author_rank", "rank", "rank_fos", "age_of_article", "references", "is_open_access_True"]]
y_author = df_train[["citations"]]

In [ ]:
X_author

In [69]:
import numpy
y_author = numpy.ravel(y_author)

In [70]:
X_t, X_v, y_t, y_v = train_test_split(X_author, y_author, test_size=0.2, random_state=42)

In [ ]:
y_t.shape

In [ ]:
y_v.shape

In [ ]:
#X_train = X_t
#X_val = X_v
#y_train = y_t
#y_val = y_v

In [ ]:
X_t.shape

In [ ]:
X_v.shape

In [71]:
ss = StandardScaler()

X_t = ss.fit_transform(X_t)
X_v = ss.transform(X_v)

In [72]:

from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_t,y_t)
y_pred = lr.predict(X_v)
score(y_v, y_pred)

0.34876084496236226

In [ ]:
#references with respect to citations
#abstract with respect to citations

In [ ]:
X_t.shape

In [ ]:
y_t.shape

In [ ]:
y_v.shape

In [ ]:
y_pred.shape

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
for passes in [1,10,30,50, 70, 90]:
    model = RandomForestRegressor(random_state = 0, n_estimators = passes)
    model.fit(X_t, y_t)
    y_pred = model.predict(X_v)
    r2 = score(y_v, y_pred)
    print("Forest tree depth = {} r2 is {}".format(passes,r2))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
for passes in [30,50,80,100]:
    model = DecisionTreeClassifier(random_state = 123, max_depth = passes)
    model.fit(X_t, y_t)
    y_pred = model.predict(X_v)
    r2 = score(y_v, y_pred)
    print("Forest tree depth = {} r2 is {}".format(passes,r2))




In [ ]:
from sklearn.linear_model import Perceptron

for passes in [8,9,10]:
    model = Perceptron(random_state = 0, max_iter = passes)
    model.fit(X_t, y_t)
    y_pred = model.predict(X_v)
    r2 = score(y_v, y_pred)
    print("Forest tree depth = {} r2 is {}".format(passes,r2))